### Libraries

In [1]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sklearn

import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from plotly.subplots import make_subplots
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Get the current working directory
directorio_actual = os.getcwd()

# Specify the relative path from the current directory
ruta_csv_relativa = os.path.join('..','data','03_primary','3.spotify.csv')

# Load the CSV file
spotify = pd.read_csv(ruta_csv_relativa)

In [3]:
spotify

,popularity,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre,duration_ms_scaled,popularity_scaled,track_genre_encoded,intensity
0,73.0,0.669059,0.460989,1,0.779395,0.128221,0.032329,0.000001,0.352006,0.718593,0.301970,acoustic,0.461947,0.920689,0,0.308429
1,55.0,0.394881,0.165983,1,0.574223,0.057482,0.927711,0.000006,0.092607,0.268342,0.243282,acoustic,0.387347,0.849624,0,0.065544
2,57.0,0.414159,0.358987,0,0.720948,0.035635,0.210843,0.000000,0.108756,0.120603,0.236770,acoustic,0.446450,0.858572,0,0.148678
3,71.0,0.229945,0.059581,0,0.549185,0.015060,0.908635,0.000071,0.123896,0.143719,0.830006,acoustic,0.439024,0.913703,0,0.013700
4,82.0,0.606940,0.442989,2,0.721984,0.032347,0.470884,0.000000,0.074338,0.167839,0.482246,acoustic,0.436375,0.949954,0,0.268868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113794,21.0,0.129271,0.234985,5,0.590693,0.021317,0.642570,0.928000,0.077769,0.034070,0.516273,world-music,0.550216,0.611400,113,0.030377
113795,22.0,0.131413,0.116982,0,0.553039,0.019090,0.997992,0.976000,0.096644,0.035176,0.286899,world-music,0.550216,0.622734,113,0.015373
113796,22.0,0.618721,0.328986,0,0.698238,0.021105,0.870482,0.000000,0.075347,0.746734,0.552197,world-music,0.490010,0.622734,113,0.203551
113797,41.0,0.573739,0.505990,7,0.698355,0.008060,0.382530,0.000000,0.263184,0.415075,0.572356,world-music,0.497723,0.776273,113,0.290306


In [4]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Definir varias combinaciones de predictores
predictors_1 = ['danceability', 'energy', 'loudness']  # Características básicas
predictors_2 = ['danceability', 'energy', 'loudness', 'speechiness', 'valence']  # Añadir más variables relacionadas con la emoción y lo hablado
predictors_3 = ['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'tempo', 'intensity']  # Incluyendo tempo e intensidad
predictors_4 = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms_scaled','track_genre_encoded', 'intensity']  # Todo el set

# Función para entrenar y obtener resultados con diferentes predictores
def entrenar_modelos(predictors):
    # Separar en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(spotify[predictors], spotify['popularity_scaled'], test_size=0.2, random_state=42)
    
    # Regresión lineal
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    
    # Hacer predicciones y calcular el error cuadrático medio
    y_pred = lin_reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    # Obtener los coeficientes de la regresión lineal
    coeficientes = lin_reg.coef_
    
    # Combinar coeficientes con los nombres de las características
    importancia_caracteristicas = pd.DataFrame({
        'Característica': predictors,
        'Importancia (Coeficiente)': coeficientes
    })
    
    # Mostrar resultados
    print(f"Resultados para predictores: {predictors}")
    print(f"MSE: {mse}, RMSE: {rmse}")
    print(importancia_caracteristicas)
    print("\n")

# Probar los diferentes conjuntos de predictores
entrenar_modelos(predictors_1)
entrenar_modelos(predictors_2)
entrenar_modelos(predictors_3)
entrenar_modelos(predictors_4)


Resultados para predictores: ['danceability', 'energy', 'loudness']
MSE: 0.029462449920644833, RMSE: 0.17164629305826803
  Característica  Importancia (Coeficiente)
0   danceability                  -0.017501
1         energy                  -0.097064
2       loudness                   0.266799


Resultados para predictores: ['danceability', 'energy', 'loudness', 'speechiness', 'valence']
MSE: 0.029331667631335632, RMSE: 0.1712649048443248
  Característica  Importancia (Coeficiente)
0   danceability                  -0.001172
1         energy                  -0.085742
2       loudness                   0.254946
3    speechiness                  -0.072149
4        valence                  -0.018191


Resultados para predictores: ['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'tempo', 'intensity']
MSE: 0.02931365868439181, RMSE: 0.1712123204807172
  Característica  Importancia (Coeficiente)
0   danceability                   0.055036
1         energy                  

In [22]:


# Definir las combinaciones recomendadas de predictores
predictors_essential = ['danceability', 'energy', 'loudness', 'speechiness', 'intensity']
predictors_simplified = ['loudness', 'intensity', 'speechiness']
predictors_moderate = ['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'intensity']
predictors_complete = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 
                       'liveness', 'valence', 'intensity']

# Función para probar modelos y agregar métricas adicionales
def probar_modelo_con_metricas(predictors, nombre_conjunto):
    X_train, X_test, y_train, y_test = train_test_split(spotify[predictors], spotify['popularity_scaled'], test_size=0.2, random_state=42)
    
    # Random Forest
    rf_model = RandomForestRegressor(n_estimators=50, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_pred)
    rf_rmse = np.sqrt(rf_mse)
    
    # Gradient Boosting
    gb_model = GradientBoostingRegressor(n_estimators=50, random_state=42)
    gb_model.fit(X_train, y_train)
    gb_pred = gb_model.predict(X_test)
    gb_mse = mean_squared_error(y_test, gb_pred)
    gb_rmse = np.sqrt(gb_mse)
    
    # Regresión Lineal
    lin_model = LinearRegression()
    lin_model.fit(X_train, y_train)
    lin_pred = lin_model.predict(X_test)
    lin_mse = mean_squared_error(y_test, lin_pred)
    lin_rmse = np.sqrt(lin_mse)
    lin_mae = mean_absolute_error(y_test, lin_pred)
    lin_r2 = r2_score(y_test, lin_pred)
    
    # Support Vector Regressor (SVR)
    svr_model = SVR(kernel='linear')
    svr_model.fit(X_train, y_train)
    svr_pred = svr_model.predict(X_test)
    svr_mse = mean_squared_error(y_test, svr_pred)
    svr_rmse = np.sqrt(svr_mse)
    
    # Resultados
    resultados = {
        'Conjunto de Predictores': nombre_conjunto,
        'Random Forest MSE': rf_mse,
        'Random Forest RMSE': rf_rmse,
        'Gradient Boosting MSE': gb_mse,
        'Gradient Boosting RMSE': gb_rmse,
        'Linear Regression MSE': lin_mse,
        'Linear Regression RMSE': lin_rmse,
        'Linear Regression MAE': lin_mae,
        'Linear Regression R²': lin_r2,
        'SVR MSE': svr_mse,
        'SVR RMSE': svr_rmse
    }
    
    return resultados

# Probar con los cuatro conjuntos de predictores
resultados_essential = probar_modelo_con_metricas(predictors_essential, 'Essential Predictors')
resultados_simplified = probar_modelo_con_metricas(predictors_simplified, 'Simplified Predictors')
resultados_moderate = probar_modelo_con_metricas(predictors_moderate, 'Moderate Predictors')
resultados_complete = probar_modelo_con_metricas(predictors_complete, 'Complete Predictors')

# Mostrar los resultados de las pruebas con diferentes conjuntos de predictores
resultados_finales = pd.DataFrame([resultados_essential, resultados_simplified, resultados_moderate, resultados_complete])
print(resultados_finales)

  Conjunto de Predictores  Random Forest MSE  Random Forest RMSE  \
0    Essential Predictors           0.020499            0.143174   
1   Simplified Predictors           0.021169            0.145496   
2     Moderate Predictors           0.019911            0.141106   
3     Complete Predictors           0.018666            0.136622   

   Gradient Boosting MSE  Gradient Boosting RMSE  Linear Regression MSE  \
0               0.028384                0.168477               0.029349   
1               0.028636                0.169221               0.029501   
2               0.028272                0.168141               0.029313   
3               0.027658                0.166306               0.028807   

   Linear Regression RMSE  Linear Regression MAE  Linear Regression R²  \
0                0.171316               0.115433              0.013413   
1                0.171760               0.116167              0.008292   
2                0.171211               0.115318             

In [23]:
resultados_finales

,Conjunto de Predictores,Random Forest MSE,Random Forest RMSE,Gradient Boosting MSE,Gradient Boosting RMSE,Linear Regression MSE,Linear Regression RMSE,Linear Regression MAE,Linear Regression R²,SVR MSE,SVR RMSE
0,Essential Predictors,0.020499,0.143174,0.028384,0.168477,0.029349,0.171316,0.115433,0.013413,0.029985,0.173161
1,Simplified Predictors,0.021169,0.145496,0.028636,0.169221,0.029501,0.171760,0.116167,0.008292,0.030023,0.173273
2,Moderate Predictors,0.019911,0.141106,0.028272,0.168141,0.029313,0.171211,0.115318,0.014617,0.029948,0.173054
3,Complete Predictors,0.018666,0.136622,0.027658,0.166306,0.028807,0.169725,0.113692,0.031655,0.029507,0.171776


### Random Forest es el mejor modelo

In [26]:
# Importar las bibliotecas necesarias
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

spotify['popular_binary'] = (spotify['popularity_scaled'] >= 0.5).astype(int)

# Definir las combinaciones recomendadas de predictores
predictors_essential = ['danceability', 'energy', 'loudness', 'speechiness', 'intensity']
predictors_simplified = ['loudness', 'intensity', 'speechiness']
predictors_moderate = ['danceability', 'energy', 'loudness', 'speechiness', 'valence', 'intensity']
predictors_complete = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 
                       'liveness', 'valence', 'intensity']

# Función para entrenar y evaluar tres modelos (Regresión Logística, SVM, Random Forest) y agregar métricas adicionales
def probar_modelo_con_metricas(predictors, nombre_conjunto):
    X_train, X_test, y_train, y_test = train_test_split(spotify[predictors], spotify['popular_binary'], test_size=0.2, random_state=42)
    
    # Regresión Logística
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(X_train, y_train)
    log_reg_pred = log_reg.predict(X_test)
    log_accuracy = accuracy_score(y_test, log_reg_pred)
    log_precision = precision_score(y_test, log_reg_pred)
    log_recall = recall_score(y_test, log_reg_pred)
    log_f1 = f1_score(y_test, log_reg_pred)
    
    # SVM
    svm_model = SVC(kernel='linear')
    svm_model.fit(X_train, y_train)
    svm_pred = svm_model.predict(X_test)
    svm_accuracy = accuracy_score(y_test, svm_pred)
    svm_precision = precision_score(y_test, svm_pred)
    svm_recall = recall_score(y_test, svm_pred)
    svm_f1 = f1_score(y_test, svm_pred)
    
    # Random Forest
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_pred)
    rf_precision = precision_score(y_test, rf_pred)
    rf_recall = recall_score(y_test, rf_pred)
    rf_f1 = f1_score(y_test, rf_pred)
    
    # Resultados
    resultados = {
        'Conjunto de Predictores': nombre_conjunto,
        'Logistic Regression Accuracy': log_accuracy,
        'Logistic Regression Precision': log_precision,
        'Logistic Regression Recall': log_recall,
        'Logistic Regression F1-Score': log_f1,
        'SVM Accuracy': svm_accuracy,
        'SVM Precision': svm_precision,
        'SVM Recall': svm_recall,
        'SVM F1-Score': svm_f1,
        'Random Forest Accuracy': rf_accuracy,
        'Random Forest Precision': rf_precision,
        'Random Forest Recall': rf_recall,
        'Random Forest F1-Score': rf_f1
    }
    
    return resultados

# Probar con los cuatro conjuntos de predictores
resultados_essential = probar_modelo_con_metricas(predictors_essential, 'Essential Predictors')
resultados_simplified = probar_modelo_con_metricas(predictors_simplified, 'Simplified Predictors')
resultados_moderate = probar_modelo_con_metricas(predictors_moderate, 'Moderate Predictors')
resultados_complete = probar_modelo_con_metricas(predictors_complete, 'Complete Predictors')

# Mostrar los resultados de las pruebas con diferentes conjuntos de predictores
resultados_finalesLogistic = pd.DataFrame([resultados_essential, resultados_simplified, resultados_moderate, resultados_complete])
print(resultados_finales)


  Conjunto de Predictores  Random Forest MSE  Random Forest RMSE  \
0    Essential Predictors           0.020499            0.143174   
1   Simplified Predictors           0.021169            0.145496   
2     Moderate Predictors           0.019911            0.141106   
3     Complete Predictors           0.018666            0.136622   

   Gradient Boosting MSE  Gradient Boosting RMSE  Linear Regression MSE  \
0               0.028384                0.168477               0.029349   
1               0.028636                0.169221               0.029501   
2               0.028272                0.168141               0.029313   
3               0.027658                0.166306               0.028807   

   Linear Regression RMSE  Linear Regression MAE  Linear Regression R²  \
0                0.171316               0.115433              0.013413   
1                0.171760               0.116167              0.008292   
2                0.171211               0.115318             

In [27]:
resultados_finalesLogistic

,Conjunto de Predictores,Logistic Regression Accuracy,Logistic Regression Precision,Logistic Regression Recall,Logistic Regression F1-Score,SVM Accuracy,SVM Precision,SVM Recall,SVM F1-Score,Random Forest Accuracy,Random Forest Precision,Random Forest Recall,Random Forest F1-Score
0,Essential Predictors,0.916916,0.916916,1.0,0.956657,0.916916,0.916916,1.0,0.956657,0.925879,0.936629,0.985864,0.960616
1,Simplified Predictors,0.916916,0.916916,1.0,0.956657,0.916916,0.916916,1.0,0.956657,0.926450,0.936746,0.986391,0.960928
2,Moderate Predictors,0.916916,0.916916,1.0,0.956657,0.916916,0.916916,1.0,0.956657,0.927153,0.936753,0.987206,0.961318
3,Complete Predictors,0.916916,0.916916,1.0,0.956657,0.916916,0.916916,1.0,0.956657,0.928339,0.938663,0.986295,0.961890


In [ ]:
### 